# Imports

In [1]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import soundfile as sf
from scipy.fft import rfft, rfftfreq
from sklearn.metrics import accuracy_score
import librosa
import resampy
import librosa.display
import IPython.display as ipd
import os
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from pydub import AudioSegment
import os
warnings.filterwarnings('ignore')


# Model

### define features extraction function

In [2]:
def mfcc_feature_extractor(audio,sampleRate):
    mfccsFeatures = librosa.feature.mfcc(y=audio, sr=sampleRate, n_mfcc=40)
    mfccsScaledFeatures = np.mean(mfccsFeatures.T,axis=0)
    
    return mfccsScaledFeatures

In [3]:
def contrast_feature_extractor(audio,sampleRate):
    stft = np.abs(librosa.stft(audio))
    contrast = librosa.feature.spectral_contrast(S=stft, sr=sampleRate) 
    contrastScaled = np.mean(contrast.T,axis=0)
    
    return contrastScaled

In [4]:
def tonnetz_feature_extractor(audio,sampleRate):
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(audio), sr=sampleRate)
    tonnetzScaled = np.mean(tonnetz.T,axis=0)
    
    return tonnetzScaled

In [5]:
def centroid_feature_extractor(audio,sampleRate):
    centroid = librosa.feature.spectral_centroid(y=audio,sr=sampleRate) 
    centroidScaled = np.mean(centroid.T,axis=0)
    
    return centroidScaled

In [6]:
def chroma_feature_extractor(audio,sampleRate):
    stft = np.abs(librosa.stft(audio))
    chroma = librosa.feature.chroma_stft(S=stft,sr=sampleRate)
    chromaScaled = np.mean(chroma.T,axis=0)
    
    return chromaScaled

In [7]:
def features_extractor(file):
    if file.lower().endswith(".m4a"):
        audio = AudioSegment.from_file(file, format="m4a")
        file_name, _ = os.path.splitext(os.path.basename(file))
        output_path = os.path.join(os.path.dirname(file), file_name + ".wav")
        audio.export(output_path, format="wav")
        os.remove(file)
        file = output_path

    features = []
    audio, sample_rate = librosa.load(file, res_type='kaiser_fast') 
    mfcc = mfcc_feature_extractor(audio, sample_rate)
    contrast = contrast_feature_extractor(audio, sample_rate)
    tonnetz = tonnetz_feature_extractor(audio, sample_rate)
    chroma = chroma_feature_extractor(audio, sample_rate)

    # Concatenate individual feature arrays
    concatenated_features = np.concatenate((mfcc, contrast, tonnetz, chroma), axis=0)
    features.append(concatenated_features)

    return features

### define add features function

In [8]:
def add_features(extractedFeatures, dirPath, label):
    for file in os.listdir(dirPath):
        filePath = os.path.join(dirPath, file)

        if filePath.lower().endswith(".m4a"):
            audio = AudioSegment.from_file(filePath, format="m4a")
            file_name, _ = os.path.splitext(os.path.basename(filePath))
            output_path = os.path.join(os.path.dirname(filePath), file_name + ".wav")
            audio.export(output_path, format="wav")
            os.remove(filePath)
            filePath = output_path

        audio, sampleRate = librosa.load(filePath, res_type='kaiser_fast') 
        mfcc = mfcc_feature_extractor(audio, sampleRate)
        contrast = contrast_feature_extractor(audio, sampleRate)
        tonnetz = tonnetz_feature_extractor(audio, sampleRate)
        chroma = chroma_feature_extractor(audio, sampleRate)

        extractedFeatures.append([mfcc, contrast, tonnetz, chroma, label])

### creating dictionary with dirPath and label

In [9]:
dict1 = {
    "ibrahim": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/ibrahim",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/ibrahim",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/ibrahim",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/ibrahim",
    ],
    "ahmeda": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/ahmeda",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/ahmeda",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/ahmeda",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/ahmeda",
    ],
    "omar": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/omar",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/omar",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/omar",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/omar",
    ],
    "hazem": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/hazem",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/hazem",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/hazem",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/hazem",
    ],
    "ahmedk": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/ahmedk",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/ahmedk",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/ahmedk",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/ahmedk",
    ],
    "hassan": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/hassan",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/hassan",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/hassan",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/hassan",
    ],
    "mohannad": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/mohannad",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/mohannad",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/mohannad",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/mohannad",
    ],
    "other":[
        # "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other people", 
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/tamer", 
    ]
}

# class imbalance

### creating data frame from our data

In [10]:
extractedFeatures = []
for label in dict1.keys():
    for link in dict1[label]:
        add_features(extractedFeatures, link, label)

In [11]:
data = pd.DataFrame(extractedFeatures, columns=['mfcc','contrast','tonnetz','chroma','class'])
data.to_csv('processing.csv')
print(data.shape)
data.describe().T

(316, 5)


,count,unique,top,freq
mfcc,316,316,"[-422.98352, 128.33788, 39.48789, 31.981009, 2...",1
contrast,316,316,"[22.726084348385594, 13.869069821878082, 16.11...",1
tonnetz,316,316,"[-0.007179744528078988, -0.005671084393048673,...",1
chroma,316,316,"[0.5809576, 0.59106517, 0.6385473, 0.599889, 0...",1
class,316,8,ahmedk,60


### splitting the data into features and target

In [12]:
features = data.iloc[:,0:-1]
target = data['class']
target.unique()

array(['ibrahim', 'ahmeda', 'omar', 'hazem', 'ahmedk', 'hassan',
       'mohannad', 'other'], dtype=object)

In [13]:
features = features.values.tolist()
for i in range(len(features)):
    features[i] = np.concatenate((features[i][0],features[i][1],features[i][2],features[i][3]))

### encoding the target

In [14]:
encoder = LabelEncoder()
target = encoder.fit_transform(target)

In [15]:
# # Assuming 'target' is your class labels
# encoder = LabelEncoder()
# target_encoded = encoder.fit_transform(target)

# # Display the mapping between original classes and encoded values
# class_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
# print("Class encoding mapping:")
# print(class_mapping)

### splitting the data into train and test

In [16]:
xTrain, xTest, yTrain, yTest = train_test_split(features, target, test_size=0.25, random_state=29)

In [17]:
for i in range(0,8,1):
  print(i,"  ", len(target[target==i]))
  # {'ahmeda': 0, 'ahmedk': 1, 'hassan': 2, 'hazem': 3, 'ibrahim': 4, 'mohannad': 5, 'omar': 6, 'other': 7}

0    34
1    60
2    32
3    48
4    47
5    33
6    43
7    19


### creating SVM model

In [18]:
# classifier = SVC(kernel='linear',decision_function_shape="ovo" ,probability=True) 
# classifier.fit(xTrain,yTrain)
# # 99

### creating XGBoost model

In [19]:
classifier = XGBClassifier(objective='multi:softmax', num_class=8, eval_metric='mlogloss')
classifier.fit(xTrain, yTrain)
# 98

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_class=8, num_parallel_tree=None, ...)

### creating RandomForest model

In [20]:
# classifier = RandomForestClassifier(n_estimators=100, random_state=42)  
# classifier.fit(xTrain, yTrain)
# # 97

### predicting xTest

In [21]:
predX = classifier.predict(xTest)

### making classification report to see the result of the predicton 

In [22]:
print(classification_report(yTest, predX))

train_predictions = classifier.predict(xTrain)
test_predictions = classifier.predict(xTest)

train_accuracy = accuracy_score(yTrain, train_predictions)
test_accuracy = accuracy_score(yTest, test_predictions)

print(train_accuracy, "  ", test_accuracy)

              precision    recall  f1-score   support

           0       1.00      0.86      0.92         7
           1       1.00      1.00      1.00        18
           2       0.80      0.67      0.73         6
           3       0.90      1.00      0.95         9
           4       1.00      1.00      1.00        15
           5       0.78      0.88      0.82         8
           6       1.00      1.00      1.00         9
           7       1.00      1.00      1.00         7

    accuracy                           0.95        79
   macro avg       0.93      0.92      0.93        79
weighted avg       0.95      0.95      0.95        79

1.0    0.9493670886075949


In [23]:
def prediction(pred):   
    # {'ahmeda': 0, 'ahmedk': 1, 'hassan': 2, 'hazem': 3, 'ibrahim': 4, 'mohannad': 5, 'omar': 6, 'other': 7}
    if pred == 0:
        return ("ahmeda")
    elif pred == 1:
        return ("ahmedk")
    elif pred == 2:
        return ("hassan")
    elif pred == 3:
        return ("hazem")
    elif pred == 4:
        return ("ibrahim")
    elif pred == 5:
        return ("mohannad")
    elif pred == 6:
        return ("omar")
    elif pred == 7:
        return ("other")

In [24]:
def my_predict(features):
    probabilities = classifier.predict_proba(features)
    # print(probabilities)

    for i in range(len(features)):
        class_pred_index = probabilities[i].argmax()  
        class_pred = probabilities[i, class_pred_index]  
        threshold = 0.75
        
        if class_pred > threshold:
            return class_pred_index
        else:
            return 7

In [25]:
for file in os.listdir("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/mohannad")[0:5]:
  testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/mohannad/{file}") 
  print(classifier.predict_proba(testFeatures))
  print("1: ",prediction(classifier.predict(testFeatures)))
  print("2: ",prediction(my_predict(testFeatures))) 

[[8.9969905e-04 1.1460753e-03 7.7363774e-03 1.1700167e-03 1.8318851e-03
  9.8435551e-01 1.7162372e-03 1.1441512e-03]]
1:  mohannad
2:  mohannad
[[1.2617501e-03 8.5647876e-04 5.6797923e-03 1.1959511e-03 2.7208212e-03
  9.8625958e-01 8.5481454e-04 1.1708129e-03]]
1:  mohannad
2:  mohannad
[[0.00547009 0.00296802 0.6304285  0.00271881 0.00555956 0.3458355
  0.00296226 0.00405731]]
1:  hassan
2:  other
[[0.00229866 0.00167339 0.00154882 0.00158336 0.00341398 0.9863517
  0.00132089 0.00180919]]
1:  mohannad
2:  mohannad
[[0.00106747 0.00116033 0.00192651 0.00178787 0.00243567 0.98887795
  0.00115808 0.00158618]]
1:  mohannad
2:  mohannad


In [26]:
for file in os.listdir("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/omar")[0:5]:
  testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/omar/{file}") 
  print(classifier.predict_proba(testFeatures))
  print("1: ",prediction(classifier.predict(testFeatures)))
  print("2: ",prediction(my_predict(testFeatures))) 

[[0.09499714 0.03429512 0.02349498 0.21130289 0.08767714 0.07646473
  0.4507424  0.02102557]]
1:  omar
2:  other
[[7.3445222e-04 2.6119235e-03 6.4561283e-04 1.6004754e-03 3.9516762e-03
  1.5472426e-03 9.8809159e-01 8.1714551e-04]]
1:  omar
2:  omar
[[9.2688715e-04 1.6820822e-03 9.2976721e-04 8.8662241e-04 2.2233289e-03
  1.2207907e-03 9.9109930e-01 1.0312471e-03]]
1:  omar
2:  omar
[[0.00138033 0.00250498 0.00138462 0.00258154 0.00196415 0.00137246
  0.98727614 0.00153575]]
1:  omar
2:  omar
[[1.8172653e-03 1.6620444e-03 6.7742891e-04 2.2262852e-03 5.7186228e-03
  1.0126892e-03 9.8613423e-01 7.5136730e-04]]
1:  omar
2:  omar


In [27]:
# for file in os.listdir("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other people")[0:5]:
#   testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other people/{file}") 
#   print(classifier.predict_proba(testFeatures))
#   print("1: ",prediction(classifier.predict(testFeatures)))
#   print("2: ",prediction(my_predict(testFeatures))) 

In [28]:
testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/dataset/recorded/1.wav") 
print(classifier.predict_proba(testFeatures))
print("1: ",prediction(classifier.predict(testFeatures)))
print("2: ",prediction(my_predict(testFeatures))) 

[[1.0658252e-03 1.6759193e-02 1.0691375e-03 1.3116521e-03 3.0982276e-02
  7.7203853e-04 9.4668460e-01 1.3554144e-03]]
1:  omar
2:  omar


In [29]:
for file in os.listdir("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/tamer")[0:5]:
  testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/tamer/{file}") 
  print(classifier.predict_proba(testFeatures))
  print("1: ",prediction(classifier.predict(testFeatures)))
  print("2: ",prediction(my_predict(testFeatures))) 

[[0.00446916 0.00915597 0.00233306 0.00507214 0.0024513  0.00204506
  0.00242022 0.97205305]]
1:  other
2:  other
[[0.00380758 0.003501   0.0017963  0.00637118 0.00379489 0.00191655
  0.00243099 0.9763815 ]]
1:  other
2:  other
[[0.00235874 0.00318873 0.00178194 0.00520463 0.01435989 0.00211866
  0.00241156 0.9685759 ]]
1:  other
2:  other
[[0.00236348 0.00762627 0.00178552 0.00927793 0.00422715 0.00178155
  0.0024164  0.9705217 ]]
1:  other
2:  other
[[0.00221729 0.00318072 0.00177747 0.00356224 0.01601678 0.00199356
  0.00510896 0.966143  ]]
1:  other
2:  other


In [30]:
pickle_out = open("processing.pkl", "wb")
pickle.dump(classifier, pickle_out)
pickle_out.close()